In [29]:
import contextlib
import os

@contextlib.contextmanager
def set_env(**environ):
    """
    Modelled after https://stackoverflow.com/questions/2059482/temporarily-modify-the-current-processs-environment

    Temporarily set the process environment variables.

    >>> with set_env(PLUGINS_DIR='test/plugins'):
    ...   "PLUGINS_DIR" in os.environ
    True

    >>> "PLUGINS_DIR" in os.environ
    False

    :type environ: dict[str, unicode]
    :param environ: Environment variables to set
    """
    old_environ = dict(os.environ)
    os.environ.update(environ)
    try:
        yield
    finally:
        os.environ.clear()
        os.environ.update(old_environ)

In [28]:

os.environ.update(TEST="2", TESTT="3")

ValueError: not enough values to unpack (expected 2, got 1)

In [26]:
os.environ["TEST"]

'2'

## Tool by tool walkthrough

A step by step runthrough of the separate veg2hab tools

In [16]:
from veg2hab.main import run
from veg2hab.io.common import (
    AccessDBInputs,
    StackVegKarteringInputs,
    ApplyDefTabelInputs,
    ApplyMozaiekInputs,
    ApplyFunctioneleSamenhangInputs,
)
from veg2hab.io.cli import CLIInterface
from veg2hab.enums import WelkeTypologie

import geopandas as gpd

CLIInterface.get_instance()

In [17]:
step_1 = AccessDBInputs(
    shapefile="../testing/vegetatiekarteringen/FR/Rottige_Meenthe_Brandemeer/Rottige_Meenthe_Brandemeer_2013/vlakken.shp",
    elmid_col="ElmID",
    access_mdb_path="../testing/vegetatiekarteringen/FR/Rottige_Meenthe_Brandemeer/Rottige_Meenthe_Brandemeer_2013/864_RottigeMeenthe2013.mdb",
    welke_typologie="SBB",
    output="../data/notebook_data/tool_by_tool/1.gpkg",
)

step_3 = ApplyDefTabelInputs(
    shapefile=str(step_1.output),
    output="../data/notebook_data/tool_by_tool/3.gpkg"
)

step_4 = ApplyMozaiekInputs(
    shapefile=str(step_3.output),
    output="../data/notebook_data/tool_by_tool/4.gpkg"
)

step_5 = ApplyFunctioneleSamenhangInputs(
    shapefile=str(step_4.output),
    output="../data/notebook_data/tool_by_tool/5.gpkg"
)

## No changes run

In [18]:
print("Starting step 1")
run(step_1)
print("Starting step 3")
run(step_3)
print("Starting step 4")
run(step_4)
print("Starting step 5")
run(step_5)
gpd.read_file(step_5.output).head()

Starting step 1


KeyError: "['hi2', 'hi'] not in index"

## With changes run

In [ ]:
print("----- Starting step 1 -----")
run(step_1)
gdf = gpd.read_file(step_1.output)
# Verkleinen voor speeeeeed
gdf = gdf.head(10)
g1 = gdf.copy()
gdf.loc[0, "EDIT_SBB2"] = "1a1a"
gdf.loc[0, "EDIT_VvN2"] = "1aa1a"
g1post = gdf.copy()
gdf.to_file(step_1.output, driver="GPKG", layer="main")
print("----- Starting step 3 -----")
run(step_3)
gdf = gpd.read_file(step_3.output)
g3 = gdf.copy()
gdf.loc[0, "EDIT_Habtype2"] = "H1234"
gdf.loc[0, "EDIT_Kwal2"] = "M"
g3post = gdf.copy()
gdf.to_file(step_3.output, driver="GPKG", layer="main")
print("----- Starting step 4 -----")
run(step_4)
gdf = gpd.read_file(step_4.output)
g4 = gdf.copy()
gdf.loc[1, "EDIT_Habtype1"] = "H4321"
gdf.loc[1, "EDIT_Kwal1"] = "G"
g4post = gdf.copy()
gdf.to_file(step_4.output, driver="GPKG", layer="main")
print("----- Starting step 5 -----")
run(step_5)
done = gpd.read_file(step_5.output)
print(done.iloc[0][["SBB1", "VvN1", "Perc1", "Habtype1", "SBB2", "VvN2", "Perc2", "Habtype2"]])
print(done.iloc[1][["SBB1", "VvN1", "Perc1", "Habtype1"]])
done.head()

## Doing illegal things

In [ ]:
print("----- Starting step 1 -----")
run(step_1)
gdf = gpd.read_file(step_1.output)
# Verkleinen voor speeeeeed
gdf = gdf.head(10)
gdf.loc[0, "EDIT_SBB3"] = "1a1a"
gdf.loc[0, "EDIT_VvN3"] = "1aa1a"
gdf.loc[0, "EDIT_perc3"] = "10"

gdf.loc[1, "EDIT_SBB2"] = "1a1a"
gdf.loc[1, "EDIT_VvN2"] = "1aa1a"
gdf.loc[1, "EDIT_perc2"] = "10"
gdf.to_file(step_1.output, driver="GPKG", layer="main")
print("----- Starting step 3 -----")
run(step_3)
gdf = gpd.read_file(step_3.output)
gdf.loc[0, "EDIT_Habtype3"] = "H1234"
gdf.loc[0, "EDIT_Kwal3"] = "M"
gdf.to_file(step_3.output, driver="GPKG", layer="main")
print("----- Starting step 4 -----")
run(step_4)
gdf = gpd.read_file(step_4.output)
gdf.loc[1, "EDIT_Habtype2"] = "H4321"
gdf.loc[1, "EDIT_Kwal2"] = "G"
gdf.loc[1, "EDIT_Habtype4"] = "H4321"
gdf.loc[1, "EDIT_Kwal4"] = "G"
gdf.to_file(step_4.output, driver="GPKG", layer="main")
print("----- Starting step 5 -----")
run(step_5)
done = gpd.read_file(step_5.output)
print(done.iloc[0].SBB3)
print(done.iloc[0].Habtype3)
print(done.iloc[1].Habtype2)
done.head()

### Demo step 2 combineer karteringen

In [ ]:
from veg2hab.enums import WelkeTypologie
from veg2hab.vegkartering import Kartering
from pathlib import Path

kart1 = Kartering.from_access_db(
    shape_path=Path(
        "../testing/vegetatiekarteringen/DR/NM_Fochteloerveen2013_14/NM_Fochteloerveen2013_14.shp"
    ),
    shape_elm_id_column="ElmID",
    access_mdb_path=Path(
        "../testing/vegetatiekarteringen/DR/NM_Fochteloerveen2013_14/digitale standaard/Fochteloerveen2013-2014.mdb"
    ),
    welke_typologie=WelkeTypologie.SBB,
)

kart2 = Kartering.from_access_db(
    shape_path=Path(
        "../testing/vegetatiekarteringen/DR/NM_SBB_Fochteloerveen_randen2016/NM_SBB_Fochteloerveen_randen2016.shp"
    ),
    shape_elm_id_column="ELMID",
    access_mdb_path=Path(
        "../testing/vegetatiekarteringen/DR/NM_SBB_Fochteloerveen_randen2016/fochteloo2016.mdb"
    ),
    welke_typologie=WelkeTypologie.SBB,
)

# checking_polygon ligt volledig binnen het te overschrijven deel van kart1
checking_polygon = kart1.gdf[kart1.gdf["ElmID"] == 6001][["geometry"]]

# Het heeft 5 intersections met z'n eigen kartering (kart1)
assert len(checking_polygon.sjoin(kart1.gdf, how="inner", predicate="intersects")) == 5
intersects_kart1 = checking_polygon.sjoin(
    kart1.gdf, how="inner", predicate="intersects"
).copy()

# En 6 met kart2
assert len(checking_polygon.sjoin(kart2.gdf, how="inner", predicate="intersects")) == 6
intersects_kart2 = checking_polygon.sjoin(
    kart2.gdf, how="inner", predicate="intersects"
).copy()

gecombineerd = Kartering.combineer_karteringen([kart1, kart2])

# En na het combineren heeft het er 6 (want kart2 heeft kart1 overschreven op en rond de locatie van checking_polygon)
assert (
    len(checking_polygon.sjoin(gecombineerd.gdf, how="inner", predicate="intersects"))
    == 6
)

# Het heeft andere intersects nu dan het had met kart1 (we checken voor Area/VegTypeInfo want ElmID is veranderd)
assert not checking_polygon.sjoin(
    gecombineerd.gdf, how="inner", predicate="intersects"
)[["Area", "VegTypeInfo"]].equals(intersects_kart1[["Area", "VegTypeInfo"]])

# En het heeft dezelfde intersects dan het had met kart2 (we checken voor Area/VegTypeInfo want ElmID is veranderd)
assert checking_polygon.sjoin(gecombineerd.gdf, how="inner", predicate="intersects")[
    ["Area", "VegTypeInfo"]
].equals(intersects_kart2[["Area", "VegTypeInfo"]])

print("We zijn alle asserts langs dus t werkt :)")

In [ ]:
# Als je m wilt bekijken (het checking_polygon/overschreven gebied zit bij ElmID 4405/4406/4407)
gecombineerd.to_editable_vegtypes().to_file(
    "../testing/combined.gpkg", driver="GPKG"
)